In [ ]:
from google.colab import drive
drive.mount('/colab/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /colab/


In [ ]:
import keras
import os 
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras import regularizers

In [ ]:
os.listdir("/colab/My Drive/")

['How to get started with Drive.pdf',
 'Colab Notebooks',
 'SelfDrivingCar',
 'Copy of model-08-0.13.h5',
 'ftrainf.csv']

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/colab/My Drive/SelfDrivingCar/Dataset/07012018.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
## Reading the data
data = pd.read_csv('/tmp/data.txt', delimiter=',', header=None)
data

,0,1
0,0.jpg 0.000000,2018-07-01 17:09:44:912
1,1.jpg 0.000000,2018-07-01 17:09:44:972
2,2.jpg 0.000000,2018-07-01 17:09:45:11
3,3.jpg 0.000000,2018-07-01 17:09:45:76
4,4.jpg 0.000000,2018-07-01 17:09:45:105
...,...,...
63820,63820.jpg -11.500000,2018-07-01 18:05:36:384
63821,63821.jpg -11.290000,2018-07-01 18:05:36:484
63822,63822.jpg -11.290000,2018-07-01 18:05:36:542
63823,63823.jpg -11.290000,2018-07-01 18:05:36:583


In [ ]:
# Removing the unwanted column in the dataframe
data.drop(axis=1, columns=1, inplace=True)

In [ ]:
data

,0
0,0.jpg 0.000000
1,1.jpg 0.000000
2,2.jpg 0.000000
3,3.jpg 0.000000
4,4.jpg 0.000000
...,...
63820,63820.jpg -11.500000
63821,63821.jpg -11.290000
63822,63822.jpg -11.290000
63823,63823.jpg -11.290000


In [ ]:
## Processing the angles to the correect form and the converting them to radians

angles = data[0].str.split(" ")
data = angles.to_list()
columns=['Image_name', 'Angle']
data_log = pd.DataFrame(data, columns=columns)
data_log

,Image_name,Angle
0,0.jpg,0.000000
1,1.jpg,0.000000
2,2.jpg,0.000000
3,3.jpg,0.000000
4,4.jpg,0.000000
...,...,...
63820,63820.jpg,-11.500000
63821,63821.jpg,-11.290000
63822,63822.jpg,-11.290000
63823,63823.jpg,-11.290000


In [ ]:
Angle=data_log.drop(axis=1, columns='Image_name', index=None)
Angle['Angle_rad']=Angle['Angle'].astype('float')
Angle

,Angle,Angle_rad
0,0.000000,0.00
1,0.000000,0.00
2,0.000000,0.00
3,0.000000,0.00
4,0.000000,0.00
...,...,...
63820,-11.500000,-11.50
63821,-11.290000,-11.29
63822,-11.290000,-11.29
63823,-11.290000,-11.29


In [ ]:
Angle['Angle_rad'] = Angle['Angle_rad']*np.pi/180

In [ ]:
print(max(Angle['Angle_rad']), min(Angle['Angle_rad']))

4.408876240916168 -5.913524693780235


In [ ]:
data_log_final = pd.concat([data_log, Angle['Angle_rad']], axis = 'columns')
data_log_final

,Image_name,Angle,Angle_rad
0,0.jpg,0.000000,0.000000
1,1.jpg,0.000000,0.000000
2,2.jpg,0.000000,0.000000
3,3.jpg,0.000000,0.000000
4,4.jpg,0.000000,0.000000
...,...,...,...
63820,63820.jpg,-11.500000,-0.200713
63821,63821.jpg,-11.290000,-0.197048
63822,63822.jpg,-11.290000,-0.197048
63823,63823.jpg,-11.290000,-0.197048


In [ ]:
data_log_final=data_log_final.drop(axis=1, columns='Angle', index=None)

In [ ]:
data_log_final

,Image_name,Angle_rad
0,0.jpg,0.000000
1,1.jpg,0.000000
2,2.jpg,0.000000
3,3.jpg,0.000000
4,4.jpg,0.000000
...,...,...
63820,63820.jpg,-0.200713
63821,63821.jpg,-0.197048
63822,63822.jpg,-0.197048
63823,63823.jpg,-0.197048


In [ ]:
## Splitting the data in train and test set

data_log_train = data_log_final[0:51060]
data_log_validate = data_log_final[51060:]

In [ ]:
## Shuffling the dataset
import random
base_shuffle_train = []
base_shuffle_val = []
for num in range(51060):
  base_shuffle_train.append(num)
for num2 in range(51060,63825):
  base_shuffle_val.append(num2)
random.shuffle(base_shuffle_val)
random.shuffle(base_shuffle_train)
print(base_shuffle_val) 
print(base_shuffle_train)

[60648, 62673, 53771, 61191, 54147, 58445, 61247, 54317, 55718, 51327, 56937, 58038, 62617, 59735, 52927, 51827, 60227, 56846, 60119, 57730, 51788, 59996, 63267, 60646, 58083, 55670, 58711, 56520, 62242, 56521, 52519, 59522, 56942, 60964, 55800, 51171, 62382, 60991, 59028, 58645, 60007, 54459, 55776, 63311, 62391, 56400, 58201, 57444, 59866, 55499, 52376, 55617, 53358, 52475, 57478, 58442, 52072, 54341, 53556, 52622, 60931, 53290, 53736, 56411, 56141, 61832, 52841, 55548, 56370, 53811, 52966, 51830, 62561, 51769, 63154, 51758, 56448, 60972, 54027, 59927, 58914, 60995, 52028, 53675, 59394, 59407, 51912, 58213, 56170, 52784, 58367, 61395, 60175, 54494, 63175, 54951, 54328, 60735, 63474, 63388, 56877, 61658, 53777, 58956, 53103, 61514, 58985, 59058, 58880, 56066, 51528, 54811, 51751, 61816, 51993, 60334, 51296, 63316, 63498, 54886, 62276, 60787, 57455, 57628, 53526, 53447, 63618, 60729, 52406, 55084, 52881, 63000, 52743, 59828, 63598, 62468, 60302, 55355, 61321, 51490, 63149, 60911, 51930

In [ ]:
data_log_train = data_log_train.reindex(index = base_shuffle_train)
data_log_validate = data_log_validate.reindex(index = base_shuffle_val)

In [ ]:
data_log_train

,Image_name,Angle_rad
0,0.jpg,0.000000
1,1.jpg,0.000000
2,2.jpg,0.000000
3,3.jpg,0.000000
4,4.jpg,0.000000
...,...,...
51055,51055.jpg,-0.022864
51056,51056.jpg,-0.022864
51057,51057.jpg,-0.022864
51058,51058.jpg,-0.022864


In [ ]:
data_log_validate


,Image_name,Angle_rad
51060,51060.jpg,-0.022864
51061,51061.jpg,-0.022864
51062,51062.jpg,-0.022864
51063,51063.jpg,-0.022864
51064,51064.jpg,-0.022864
...,...,...
63820,63820.jpg,-0.200713
63821,63821.jpg,-0.197048
63822,63822.jpg,-0.197048
63823,63823.jpg,-0.197048


In [ ]:
## Using IMAGE DATA GENERATORS
IMAGES_DIR = '/tmp/data/'

training_datagen = ImageDataGenerator(
        rescale = 1./255,)

train_generator = training_datagen.flow_from_dataframe(
        dataframe=data_log_train,
        directory=IMAGES_DIR,
        x_col="Image_name",
        y_col="Angle_rad",
        target_size=(224, 224),
        batch_size=64,
        color_mode = 'grayscale',
        class_mode='other')

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
        dataframe=data_log_validate,
        directory=IMAGES_DIR,
        x_col="Image_name",
        y_col="Angle_rad",
        target_size=(224, 224),
        batch_size = 64,
        color_mode = 'grayscale',
        class_mode='other')

Found 51060 validated image filenames.
Found 12765 validated image filenames.


In [ ]:
##Call backs for early stopping and model checkpoint
import keras
import tensorflow as tf

filepath = '/colab/My Drive/SelfDrivingCar/model-{epoch:02d}-{val_loss:.2f}.h5'
model_callbacks =[
  keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
  keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1 ),
  keras.callbacks.CSVLogger('model-logs.csv', separator=',', append=False )
]

In [ ]:
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization, Lambda
from keras.models import Sequential
from keras import regularizers
from keras.optimizers import Adam

In [ ]:
'''
model = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3), kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    # Only 1 output neuron.
    tf.keras.layers.Dense(1, activation='tanh', kernel_regularizer=regularizers.l2(0.001))
])
'''

model = tf.keras.models.Sequential([
tf.keras.layers.Lambda(lambda x: x/127.5-1.0, input_shape = (224,224,3) ),
tf.keras.layers.Conv2D(24, (5, 5), activation = 'elu', strides = (2, 2) ),
tf.keras.layers.Conv2D(36, (5, 5), activation = 'elu', strides = (2, 2) ),
tf.keras.layers.Conv2D(48, (5, 5), activation = 'elu', strides = (2, 2) ),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'elu' ),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'elu' ),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(100, activation='elu'),
tf.keras.layers.Dense(50, activation='elu'),
tf.keras.layers.Dense(10, activation='elu'),
tf.keras.layers.Dense(1, activation = 'linear')
])
'''
model = tf.keras.models.Sequential([
tf.keras.layers.Lambda(lambda x: x/127.5-1.0, input_shape = (224,224,3) ),
tf.keras.layers.Conv2D(24, (5, 5), activation = 'elu', strides = (2, 2), kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.Conv2D(36, (5, 5), activation = 'elu', strides = (1, 1), kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(48, (5, 5), activation = 'elu', strides = (2, 2), kernel_regularizer=regularizers.l2(0.001) ),

tf.keras.layers.Conv2D(64, (3, 3), activation = 'elu',  kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'elu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.4),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(100, activation='elu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dense(50, activation='elu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dense(10, activation='elu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dense(1, activation = 'linear', kernel_regularizer=regularizers.l2(0.001))
])
'''
'''
model = tf.keras.models.Sequential([
tf.keras.layers.Lambda(lambda x: x/127.5-1.0, input_shape = (224,224,3) ),
tf.keras.layers.Conv2D(24, (5, 5), activation = 'elu', strides = (2, 2), kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.Conv2D(36, (5, 5), activation = 'elu', strides = (1, 1), kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(48, (5, 5), activation = 'elu', strides = (2, 2), kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'elu',  kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'elu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(100, activation='elu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(50, activation='elu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(10, activation='elu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dense(1, activation = 'linear', kernel_regularizer=regularizers.l2(0.001))
])
'''

"\nmodel = tf.keras.models.Sequential([\ntf.keras.layers.Lambda(lambda x: x/127.5-1.0, input_shape = (224,224,3) ),\ntf.keras.layers.Conv2D(24, (5, 5), activation = 'elu', strides = (2, 2), kernel_regularizer=regularizers.l2(0.001) ),\ntf.keras.layers.Conv2D(36, (5, 5), activation = 'elu', strides = (1, 1), kernel_regularizer=regularizers.l2(0.001) ),\ntf.keras.layers.MaxPooling2D(2,2),\ntf.keras.layers.Conv2D(48, (5, 5), activation = 'elu', strides = (2, 2), kernel_regularizer=regularizers.l2(0.001) ),\ntf.keras.layers.Conv2D(64, (3, 3), activation = 'elu',  kernel_regularizer=regularizers.l2(0.001)),\ntf.keras.layers.Conv2D(64, (3, 3), activation = 'elu', kernel_regularizer=regularizers.l2(0.001)),\ntf.keras.layers.Dropout(0.5),\ntf.keras.layers.Flatten(),\ntf.keras.layers.Dense(100, activation='elu', kernel_regularizer=regularizers.l2(0.001)),\ntf.keras.layers.Dropout(0.2),\ntf.keras.layers.Dense(50, activation='elu', kernel_regularizer=regularizers.l2(0.001)),\ntf.keras.layers.Drop

In [ ]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_9 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 110, 110, 24)      1824      
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 53, 53, 36)        21636     
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 25, 25, 48)        43248     
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 23, 23, 64)        27712     
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 21, 21, 64)        36928     
_________________________________________________________________
dropout_19 (Dropout)         (None, 21, 21, 64)       

In [ ]:
model.compile(loss='mse',
              optimizer='adam',
              metrics=['mae'])

In [ ]:
keras.__version__

'2.4.3'

In [ ]:
history = model.fit(
      train_generator,
        
      epochs=30,
      verbose=1,
      validation_data = val_generator,
      
      callbacks=model_callbacks)

Epoch 1/30
798/798 [==============================] - ETA: 0s - loss: 3.6867 - mae: 0.4570
Epoch 00001: val_loss improved from 0.44788 to 0.09283, saving model to /colab/My Drive/SelfDrivingCar/model-01-0.09.h5
798/798 [==============================] - 201s 252ms/step - loss: 3.6867 - mae: 0.4570 - val_loss: 0.0928 - val_mae: 0.1428
Epoch 2/30
798/798 [==============================] - ETA: 0s - loss: 0.2604 - mae: 0.2778
Epoch 00002: val_loss did not improve from 0.09283
798/798 [==============================] - 200s 251ms/step - loss: 0.2604 - mae: 0.2778 - val_loss: 0.0991 - val_mae: 0.1753
Epoch 3/30
798/798 [==============================] - ETA: 0s - loss: 12957.3643 - mae: 10.8755
Epoch 00003: val_loss did not improve from 0.09283
798/798 [==============================] - 199s 250ms/step - loss: 12957.3643 - mae: 10.8755 - val_loss: 103.3783 - val_mae: 10.1627
Epoch 4/30
798/798 [==============================] - ETA: 0s - loss: 4.5591 - mae: 1.5877
Epoch 00004: val_loss did 

In [ ]:
model.save('/colab/My Drive/SelfDrivingCar/Dataset/model-final-save.h5')

MODEL 2

In [ ]:
model2 = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(24, (5, 5), activation = 'elu', strides = (2, 2), kernel_regularizer=regularizers.l2(0.001), input_shape = (224, 224,3) ),
tf.keras.layers.Conv2D(36, (5, 5), activation = 'elu', strides = (1, 1), kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(48, (5, 5), activation = 'elu', strides = (2, 2), kernel_regularizer=regularizers.l2(0.001) ),

tf.keras.layers.Conv2D(64, (3, 3), activation = 'elu',  kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'elu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.3),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(256, activation='elu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(128, activation='elu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(64, activation='elu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(1, activation = 'linear', kernel_regularizer=regularizers.l2(0.001))
])


NameError: ignored

In [ ]:
model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 110, 110, 24)      1824      
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 106, 106, 36)      21636     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 53, 53, 36)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 25, 25, 48)        43248     
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 23, 23, 64)        27712     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 21, 21, 64)        36928     
_________________________________________________________________
dropout_8 (Dropout)          (None, 21, 21, 64)       

In [ ]:
import keras
import tensorflow as tf

filepath = '/colab/My Drive/SelfDrivingCar/model-{epoch:02d}-{val_loss:.2f}.h5'
model2_callbacks =[
  keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
  keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1 ),
  keras.callbacks.CSVLogger('model-logs.csv', separator=',', append=False )
]

In [ ]:
model2.compile(loss='mse',
              optimizer='adam',
              metrics=['mae'])

In [ ]:
history = model2.fit(
      train_generator,
        
      epochs=30,
      verbose=1,
      validation_data = val_generator,
      
      callbacks=model2_callbacks)

Epoch 1/30
798/798 [==============================] - ETA: 0s - loss: 3.0531 - mae: 0.6810
Epoch 00001: val_loss improved from inf to 1.04045, saving model to /colab/My Drive/SelfDrivingCar/model-01-1.04.h5
798/798 [==============================] - 230s 288ms/step - loss: 3.0531 - mae: 0.6810 - val_loss: 1.0405 - val_mae: 0.1409
Epoch 2/30
798/798 [==============================] - ETA: 0s - loss: 1.2014 - mae: 0.4072
Epoch 00002: val_loss improved from 1.04045 to 0.82077, saving model to /colab/My Drive/SelfDrivingCar/model-02-0.82.h5
798/798 [==============================] - 231s 289ms/step - loss: 1.2014 - mae: 0.4072 - val_loss: 0.8208 - val_mae: 0.1344
Epoch 3/30
798/798 [==============================] - ETA: 0s - loss: 0.9850 - mae: 0.3724
Epoch 00003: val_loss improved from 0.82077 to 0.68534, saving model to /colab/My Drive/SelfDrivingCar/model-03-0.69.h5
798/798 [==============================] - 231s 289ms/step - loss: 0.9850 - mae: 0.3724 - val_loss: 0.6853 - val_mae: 0.1

In [ ]:
model2.save('/colab/My Drive/SelfDrivingCar/Dataset/model2-final-save.h5')

MODEL 3

In [ ]:
model3 = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='selu', input_shape=(224, 224, 3), kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(48, (3,3), activation='selu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='selu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='selu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='selu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='selu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='selu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='selu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    # Only 1 output neuron.
    tf.keras.layers.Dense(1, activation='linear', kernel_regularizer=regularizers.l2(0.001))
])

In [ ]:
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 109, 109, 48)      13872     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 48)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 52, 52, 64)        27712     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 24, 24, 64)       

In [ ]:
import keras
import tensorflow as tf

filepath = '/colab/My Drive/SelfDrivingCar/model-{epoch:02d}-{val_loss:.2f}.h5'
model3_callbacks =[
  keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
  keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1 ),
  keras.callbacks.CSVLogger('model-logs.csv', separator=',', append=False )
]

In [ ]:
model3.compile(loss='mse',
              optimizer='adam',
              metrics=['mae'])

In [ ]:
history3 = model3.fit(
      train_generator,
        
      epochs=30,
      verbose=1,
      validation_data = val_generator,
      
      callbacks=model3_callbacks)

Epoch 1/30
798/798 [==============================] - ETA: 0s - loss: 2.9408 - mae: 0.5038
Epoch 00001: val_loss improved from inf to 1.53960, saving model to /colab/My Drive/SelfDrivingCar/model-01-1.54.h5
798/798 [==============================] - 220s 276ms/step - loss: 2.9408 - mae: 0.5038 - val_loss: 1.5396 - val_mae: 0.1557
Epoch 2/30
798/798 [==============================] - ETA: 0s - loss: 1.5709 - mae: 0.2582
Epoch 00002: val_loss improved from 1.53960 to 1.39051, saving model to /colab/My Drive/SelfDrivingCar/model-02-1.39.h5
798/798 [==============================] - 218s 273ms/step - loss: 1.5709 - mae: 0.2582 - val_loss: 1.3905 - val_mae: 0.1673
Epoch 3/30
798/798 [==============================] - ETA: 0s - loss: 1.3475 - mae: 0.2231
Epoch 00003: val_loss improved from 1.39051 to 1.23039, saving model to /colab/My Drive/SelfDrivingCar/model-03-1.23.h5
798/798 [==============================] - 212s 266ms/step - loss: 1.3475 - mae: 0.2231 - val_loss: 1.2304 - val_mae: 0.2

In [ ]:
model3.save('/colab/My Drive/SelfDrivingCar/Dataset/model3-final-save.h5')

MODEL 4

In [ ]:
model4 = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3), kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(48, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    # Only 1 output neuron.
    tf.keras.layers.Dense(1, activation='linear', kernel_regularizer=regularizers.l2(0.001))
])

In [ ]:
model4.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_90 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 109, 109, 48)      13872     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 54, 54, 48)        0         
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 52, 52, 64)        27712     
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_93 (Conv2D)           (None, 24, 24, 64)      

In [ ]:
import keras
import tensorflow as tf

filepath = '/colab/My Drive/SelfDrivingCar/model-{epoch:02d}-{val_loss:.2f}.h5'
model4_callbacks =[
  keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
  keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1 ),
  keras.callbacks.CSVLogger('model-logs.csv', separator=',', append=False )
]

In [ ]:
model4.compile(loss='mse',
              optimizer='adam',
              metrics=['mae'])

In [ ]:
history4 = model4.fit(
      train_generator,
        
      epochs=30,
      verbose=1,
      validation_data = val_generator,
      
      callbacks=model4_callbacks)

Epoch 1/30
798/798 [==============================] - ETA: 0s - loss: 0.3225 - mae: 0.2439
Epoch 00001: val_loss improved from inf to 0.09081, saving model to /colab/My Drive/SelfDrivingCar/model-01-0.09.h5
798/798 [==============================] - 206s 258ms/step - loss: 0.3225 - mae: 0.2439 - val_loss: 0.0908 - val_mae: 0.1355
Epoch 2/30
798/798 [==============================] - ETA: 0s - loss: 0.2471 - mae: 0.2434
Epoch 00002: val_loss did not improve from 0.09081
798/798 [==============================] - 206s 258ms/step - loss: 0.2471 - mae: 0.2434 - val_loss: 0.0911 - val_mae: 0.1368
Epoch 3/30
798/798 [==============================] - ETA: 0s - loss: 0.2470 - mae: 0.2434
Epoch 00003: val_loss improved from 0.09081 to 0.09045, saving model to /colab/My Drive/SelfDrivingCar/model-03-0.09.h5
798/798 [==============================] - 211s 264ms/step - loss: 0.2470 - mae: 0.2434 - val_loss: 0.0904 - val_mae: 0.1355
Epoch 4/30
798/798 [==============================] - ETA: 0s - l

In [ ]:
model4.save('/colab/My Drive/SelfDrivingCar/Dataset/model4-final-save.h5')

RESIZING TO 300, 300

In [ ]:
IMAGES_DIR = '/tmp/data/'

training_datagen = ImageDataGenerator(
        rescale = 1./255,)

train_generator2 = training_datagen.flow_from_dataframe(
        dataframe=data_log_train,
        directory=IMAGES_DIR,
        x_col="Image_name",
        y_col="Angle_rad",
        target_size=(150, 150),
        batch_size=64,
        class_mode='other')

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator2 = validation_datagen.flow_from_dataframe(
        dataframe=data_log_validate,
        directory=IMAGES_DIR,
        x_col="Image_name",
        y_col="Angle_rad",
        target_size=(150, 150),
        batch_size = 64,
        class_mode='other')

Found 51060 validated image filenames.
Found 12765 validated image filenames.


In [ ]:
model5 = tf.keras.models.Sequential([
tf.keras.layers.Lambda(lambda x: x/127.5-1.0, input_shape = (300,300,3) ),
tf.keras.layers.Conv2D(24, (5, 5), activation = 'elu', strides = (2, 2) ),
tf.keras.layers.Conv2D(36, (5, 5), activation = 'elu', strides = (2, 2) ),
tf.keras.layers.Conv2D(48, (5, 5), activation = 'elu', strides = (2, 2) ),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'elu' ),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'elu' ),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(100, activation='elu'),
tf.keras.layers.Dense(50, activation='elu'),
tf.keras.layers.Dense(10, activation='elu'),
tf.keras.layers.Dense(1, activation = 'linear')
])

In [ ]:
model5.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_16 (Lambda)           (None, 300, 300, 3)       0         
_________________________________________________________________
conv2d_105 (Conv2D)          (None, 148, 148, 24)      1824      
_________________________________________________________________
conv2d_106 (Conv2D)          (None, 72, 72, 36)        21636     
_________________________________________________________________
conv2d_107 (Conv2D)          (None, 34, 34, 48)        43248     
_________________________________________________________________
conv2d_108 (Conv2D)          (None, 32, 32, 64)        27712     
_________________________________________________________________
conv2d_109 (Conv2D)          (None, 30, 30, 64)        36928     
_________________________________________________________________
dropout_41 (Dropout)         (None, 30, 30, 64)      

In [ ]:
import keras
import tensorflow as tf

filepath = '/colab/My Drive/SelfDrivingCar/model-{epoch:02d}-{val_loss:.2f}.h5'
model5_callbacks =[
  keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
  keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1 ),
  keras.callbacks.CSVLogger('model-logs.csv', separator=',', append=False )
]

In [ ]:
model5.compile(loss='mse',
              optimizer='adam',
              metrics=['mae'])

In [ ]:
history5 = model5.fit(
      train_generator2,
        
      epochs=30,
      verbose=1,
      validation_data = val_generator2,
      
      callbacks=model5_callbacks)

Epoch 1/30
798/798 [==============================] - ETA: 0s - loss: 9.2865 - mae: 0.8750
Epoch 00001: val_loss improved from inf to 0.21042, saving model to /colab/My Drive/SelfDrivingCar/model-01-0.21.h5
798/798 [==============================] - 250s 313ms/step - loss: 9.2865 - mae: 0.8750 - val_loss: 0.2104 - val_mae: 0.3693
Epoch 2/30
798/798 [==============================] - ETA: 0s - loss: 0.3818 - mae: 0.4223
Epoch 00002: val_loss improved from 0.21042 to 0.09059, saving model to /colab/My Drive/SelfDrivingCar/model-02-0.09.h5
798/798 [==============================] - 248s 311ms/step - loss: 0.3818 - mae: 0.4223 - val_loss: 0.0906 - val_mae: 0.1430
Epoch 3/30
798/798 [==============================] - ETA: 0s - loss: 0.3302 - mae: 0.3738
Epoch 00003: val_loss did not improve from 0.09059
798/798 [==============================] - 248s 311ms/step - loss: 0.3302 - mae: 0.3738 - val_loss: 0.1497 - val_mae: 0.2833
Epoch 4/30
798/798 [==============================] - ETA: 0s - l

In [ ]:
model5.save('/colab/My Drive/SelfDrivingCar/Dataset/model5-final-save.h5')

MODEL 6

In [ ]:
model6 = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(32, (5,5), activation='relu', input_shape=(300, 300, 3), kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(48, (5,5), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (5,5), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    # Only 1 output neuron.
    tf.keras.layers.Dense(1, activation='linear', kernel_regularizer=regularizers.l2(0.001))
])

In [ ]:
model6.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_110 (Conv2D)          (None, 296, 296, 32)      2432      
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_111 (Conv2D)          (None, 144, 144, 48)      38448     
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 72, 72, 48)        0         
_________________________________________________________________
conv2d_112 (Conv2D)          (None, 68, 68, 64)        76864     
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 34, 34, 64)        0         
_________________________________________________________________
conv2d_113 (Conv2D)          (None, 32, 32, 64)      

In [ ]:
import keras
import tensorflow as tf

filepath = '/colab/My Drive/SelfDrivingCar/model-{epoch:02d}-{val_loss:.2f}.h5'
model6_callbacks =[
  keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
  keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1 ),
  keras.callbacks.CSVLogger('model-logs.csv', separator=',', append=False )
]

In [ ]:
model6.compile(loss='mse',
              optimizer='adam',
              metrics=['mae'])

In [ ]:
history6 = model6.fit(
      train_generator2,
        
      epochs=30,
      verbose=1,
      validation_data = val_generator2,
      
      callbacks=model6_callbacks)

Epoch 1/30
798/798 [==============================] - ETA: 0s - loss: 0.4048 - mae: 0.2447
Epoch 00001: val_loss improved from inf to 0.11471, saving model to /colab/My Drive/SelfDrivingCar/model-01-0.11.h5
798/798 [==============================] - 270s 338ms/step - loss: 0.4048 - mae: 0.2447 - val_loss: 0.1147 - val_mae: 0.1356
Epoch 2/30
798/798 [==============================] - ETA: 0s - loss: 0.2580 - mae: 0.2434
Epoch 00002: val_loss improved from 0.11471 to 0.09442, saving model to /colab/My Drive/SelfDrivingCar/model-02-0.09.h5
798/798 [==============================] - 267s 335ms/step - loss: 0.2580 - mae: 0.2434 - val_loss: 0.0944 - val_mae: 0.1352
Epoch 3/30
798/798 [==============================] - ETA: 0s - loss: 0.2491 - mae: 0.2434
Epoch 00003: val_loss improved from 0.09442 to 0.09123, saving model to /colab/My Drive/SelfDrivingCar/model-03-0.09.h5
798/798 [==============================] - 266s 334ms/step - loss: 0.2491 - mae: 0.2434 - val_loss: 0.0912 - val_mae: 0.1

In [ ]:
model6.save('/colab/My Drive/SelfDrivingCar/Dataset/model6-final-save.h5')

MODEL 7

In [ ]:
import keras
import tensorflow as tf

filepath = '/colab/My Drive/SelfDrivingCar/model-{epoch:02d}-{val_loss:.2f}.h5'
model7_callbacks =[
  keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
  keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1 ),
  keras.callbacks.CSVLogger('model-logs.csv', separator=',', append=False )
]

In [ ]:
model7 = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(24, (5, 5), activation = 'relu', strides = (2, 2), input_shape = (300,300,3), kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.Conv2D(36, (5, 5), activation = 'relu', strides = (2, 2), kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.Conv2D(48, (5, 5), activation = 'relu', strides = (2, 2), kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.3),
tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.3),
tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(32, activation = 'relu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(1, activation = 'linear',kernel_regularizer=regularizers.l2(0.001))
])

In [ ]:
model7.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 148, 148, 24)      1824      
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 72, 72, 36)        21636     
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 34, 34, 48)        43248     
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 32, 32, 64)        27712     
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 30, 30, 64)        36928     
_________________________________________________________________
dropout_35 (Dropout)         (None, 30, 30, 64)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 57600)            

In [ ]:
model7.compile(loss='mse',
              optimizer='adam',
              metrics=['mae'])

In [ ]:
history7 = model7.fit(
      train_generator2,
        
      epochs=30,
      verbose=1,
      validation_data = val_generator2,
      
      callbacks=model7_callbacks)

Epoch 1/30
798/798 [==============================] - ETA: 0s - loss: 0.3166 - mae: 0.2440
Epoch 00001: val_loss improved from inf to 0.09567, saving model to /colab/My Drive/SelfDrivingCar/model-01-0.10.h5
798/798 [==============================] - 278s 348ms/step - loss: 0.3166 - mae: 0.2440 - val_loss: 0.0957 - val_mae: 0.1346
Epoch 2/30
798/798 [==============================] - ETA: 0s - loss: 0.2492 - mae: 0.2434
Epoch 00002: val_loss improved from 0.09567 to 0.09097, saving model to /colab/My Drive/SelfDrivingCar/model-02-0.09.h5
798/798 [==============================] - 281s 352ms/step - loss: 0.2492 - mae: 0.2434 - val_loss: 0.0910 - val_mae: 0.1351
Epoch 3/30
798/798 [==============================] - ETA: 0s - loss: 0.2473 - mae: 0.2434
Epoch 00003: val_loss improved from 0.09097 to 0.09015, saving model to /colab/My Drive/SelfDrivingCar/model-03-0.09.h5
798/798 [==============================] - 282s 353ms/step - loss: 0.2473 - mae: 0.2434 - val_loss: 0.0901 - val_mae: 0.1

NEW MODEL

In [ ]:
'''
model8 = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(36, (5, 5), activation = 'relu', strides = (2, 2), input_shape = (224,224,3)),
tf.keras.layers.Conv2D(48, (5, 5), activation = 'relu', strides = (2, 2)),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', strides = (2, 2)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(128, activation='relu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(64, activation='relu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(32, activation = 'relu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(1, activation = 'linear')
])

from keras.applications.resnet50 import ResNet50
from tensorflow.keras import Model
# load model
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# example of loading the inception v3 model
# from keras.applications.inception_v3 import InceptionV3
# load model
# model = InceptionV3(include_top=False, input_shape=(224, 224, 3))
# summarize the model
model.summary()


for layer in model.layers:
  layer.trainable = False
y = model.output
y = Flatten()(y)
y = Dropout(0.5)(y)
y = Dense(128, activation = 'relu')(y)
y = Dropout(0.2)(y)
y = Dense(64, activation = 'relu')(y)
y = Dropout(0.2)(y)
y = Dense(1, activation='linear')(y)
model = Model(inputs=model.input, outputs=y)

model = Sequential()
model.add(Conv2D(24,[5,5],activation='relu',input_shape=[224,224,3],strides=[2,2],kernel_regularizer=regularizers.l2(0.001)))
model.add(Conv2D(36,[5,5],activation='relu',strides = [2,2],kernel_regularizer=regularizers.l2(0.001)))
model.add(Conv2D(48,[5,5],activation='relu',strides = [2,2],kernel_regularizer=regularizers.l2(0.001)))
model.add(Conv2D(64,[3,3],activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Conv2D(64,[3,3],activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Flatten())
model.add(Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(24,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(1,activation='linear',kernel_regularizer=regularizers.l2(0.001)))
'''
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(36, (5, 5), activation = 'selu', strides = (2, 2), kernel_regularizer=regularizers.l2(0.001), input_shape = (224,224,3) ),
tf.keras.layers.Conv2D(48, (5, 5), activation = 'selu', strides = (2, 2), kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'selu', kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'selu', kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(128, activation='selu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(64, activation='selu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(1, activation = 'linear',kernel_regularizer=regularizers.l2(0.001))
])

In [ ]:
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(36, (5, 5), activation = 'selu', strides = (1, 1), kernel_regularizer=regularizers.l2(0.001), input_shape = (224,224,3) ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(48, (5, 5), activation = 'selu', strides = (1, 1), kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'selu', kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'selu', kernel_regularizer=regularizers.l2(0.001) ),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(128, activation='selu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(64, activation='selu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(1, activation = 'linear',kernel_regularizer=regularizers.l2(0.001))
])

In [ ]:
#  MUCH BETTER
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(36, (5, 5), activation = 'elu', strides = (1, 1), input_shape = (224,224,3) ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(48, (5, 5), activation = 'elu', strides = (1, 1), ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'elu' ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(128, (3, 3), activation = 'elu', ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(256, (3, 3), activation = 'elu', ),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(256, activation='elu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(128, activation='elu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(64, activation='elu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(1, activation = 'linear')
])

In [ ]:
# Much better 2
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(36, (5, 5), activation = 'elu', strides = (1, 1), input_shape = (224,224,3) ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(48, (5, 5), activation = 'elu', strides = (1, 1), ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'elu' ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(128, (3, 3), activation = 'elu', ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(256, (3, 3), activation = 'elu', ),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(512, activation='elu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(256, activation='elu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(128, activation='elu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(64, activation='elu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(1, activation = 'linear')
])

In [ ]:
# best (06) model val loss
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(36, (5, 5), activation = 'elu', strides = (1, 1), input_shape = (224,224,3) ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(48, (5, 5), activation = 'elu', strides = (1, 1), ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'elu' ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(128, (3, 3), activation = 'elu', ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(256, (3, 3), activation = 'elu', ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(1024, (3, 3), activation = 'elu', ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(512, activation='elu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(256, activation='elu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(128, activation='elu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(1, activation = 'linear')
])

In [ ]:
# bestest ever
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(36, (5, 5), activation = 'elu', strides = (1, 1), input_shape = (224,224,3) ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(48, (5, 5), activation = 'elu', strides = (1, 1), ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3, 3), activation = 'elu' ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(128, (3, 3), activation = 'elu', ),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(256, (3, 3), activation = 'elu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(1024, (3, 3), activation = 'elu', kernel_regularizer=regularizers.l2(0.001)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(512, activation='elu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(256, activation='elu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(128, activation='elu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(1, activation = 'linear')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 220, 220, 36)      2736      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 36)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 106, 106, 48)      43248     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 48)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 51, 51, 64)        27712     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 23, 23, 128)       7

In [ ]:
import keras
import tensorflow as tf

filepath = '/colab/My Drive/SelfDrivingCar/model-{epoch:02d}-{val_loss:.2f}.h5'
model_callbacks =[
  keras.callbacks.EarlyStopping(monitor='val_loss', patience=8),
  keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=True, mode='auto', period=1 ),
  keras.callbacks.CSVLogger('model-logs.csv', separator=',', append=False )
]

In [ ]:
model.compile(loss='mse',
              optimizer='adam',
              metrics=['mae'])

In [ ]:
# MUCH BETTER (connect) train
history8 = model.fit(
      train_generator,
        
      epochs=30,
      verbose=1,
      validation_data = val_generator,
      
      callbacks=model_callbacks)

Epoch 1/30
798/798 [==============================] - ETA: 0s - loss: 0.5726 - mae: 0.3486
Epoch 00001: saving model to /colab/My Drive/SelfDrivingCar/model-01-0.10.h5
798/798 [==============================] - 216s 270ms/step - loss: 0.5726 - mae: 0.3486 - val_loss: 0.1000 - val_mae: 0.1763
Epoch 2/30
798/798 [==============================] - ETA: 0s - loss: 0.1489 - mae: 0.2243
Epoch 00002: saving model to /colab/My Drive/SelfDrivingCar/model-02-0.12.h5
798/798 [==============================] - 214s 269ms/step - loss: 0.1489 - mae: 0.2243 - val_loss: 0.1232 - val_mae: 0.2130
Epoch 3/30
798/798 [==============================] - ETA: 0s - loss: 0.1119 - mae: 0.2113
Epoch 00003: saving model to /colab/My Drive/SelfDrivingCar/model-03-0.11.h5
798/798 [==============================] - 215s 269ms/step - loss: 0.1119 - mae: 0.2113 - val_loss: 0.1062 - val_mae: 0.1919
Epoch 4/30
798/798 [==============================] - ETA: 0s - loss: 0.0796 - mae: 0.1892
Epoch 00004: saving model to /

In [ ]:
# MUCH BETTER 2 (connect) train
history8 = model.fit(
      train_generator,
        
      epochs=30,
      verbose=1,
      validation_data = val_generator,
      
      callbacks=model_callbacks)

Epoch 1/30
798/798 [==============================] - ETA: 0s - loss: 1.2217 - mae: 0.6206
Epoch 00001: saving model to /colab/My Drive/SelfDrivingCar/model-01-0.09.h5
798/798 [==============================] - 215s 269ms/step - loss: 1.2217 - mae: 0.6206 - val_loss: 0.0907 - val_mae: 0.1417
Epoch 2/30
798/798 [==============================] - ETA: 0s - loss: 0.3266 - mae: 0.3701
Epoch 00002: saving model to /colab/My Drive/SelfDrivingCar/model-02-0.09.h5
798/798 [==============================] - 217s 272ms/step - loss: 0.3266 - mae: 0.3701 - val_loss: 0.0871 - val_mae: 0.1308
Epoch 3/30
798/798 [==============================] - ETA: 0s - loss: 0.2571 - mae: 0.3072
Epoch 00003: saving model to /colab/My Drive/SelfDrivingCar/model-03-0.10.h5
798/798 [==============================] - 217s 272ms/step - loss: 0.2571 - mae: 0.3072 - val_loss: 0.1027 - val_mae: 0.1787
Epoch 4/30
798/798 [==============================] - ETA: 0s - loss: 0.1569 - mae: 0.2452
Epoch 00004: saving model to /

In [ ]:
# Best (06) waala
history8 = model.fit(
      train_generator,
        
      epochs=30,
      verbose=1,
      validation_data = val_generator,
      
      callbacks=model_callbacks)

Epoch 1/30
798/798 [==============================] - ETA: 0s - loss: 0.1866 - mae: 0.2337
Epoch 00001: saving model to /colab/My Drive/SelfDrivingCar/model-01-0.12.h5
798/798 [==============================] - 217s 272ms/step - loss: 0.1866 - mae: 0.2337 - val_loss: 0.1190 - val_mae: 0.2214
Epoch 2/30
798/798 [==============================] - ETA: 0s - loss: 0.0843 - mae: 0.1849
Epoch 00002: saving model to /colab/My Drive/SelfDrivingCar/model-02-0.14.h5
798/798 [==============================] - 217s 272ms/step - loss: 0.0843 - mae: 0.1849 - val_loss: 0.1438 - val_mae: 0.2464
Epoch 3/30
798/798 [==============================] - ETA: 0s - loss: 0.0456 - mae: 0.1448
Epoch 00003: saving model to /colab/My Drive/SelfDrivingCar/model-03-0.11.h5
798/798 [==============================] - 216s 271ms/step - loss: 0.0456 - mae: 0.1448 - val_loss: 0.1136 - val_mae: 0.1912
Epoch 4/30
798/798 [==============================] - ETA: 0s - loss: 0.0393 - mae: 0.1334
Epoch 00004: saving model to /

KeyboardInterrupt: ignored

In [ ]:
# bestest ever
history8 = model.fit(
      train_generator,
        
      epochs=30,
      verbose=1,
      validation_data = val_generator,
      
      callbacks=model_callbacks)

Epoch 1/30
798/798 [==============================] - ETA: 0s - loss: 0.9169 - mae: 0.3181
Epoch 00001: saving model to /colab/My Drive/SelfDrivingCar/model-01-0.35.h5
798/798 [==============================] - 217s 272ms/step - loss: 0.9169 - mae: 0.3181 - val_loss: 0.3528 - val_mae: 0.1474
Epoch 2/30
798/798 [==============================] - ETA: 0s - loss: 0.3215 - mae: 0.2141
Epoch 00002: saving model to /colab/My Drive/SelfDrivingCar/model-02-0.25.h5
798/798 [==============================] - 217s 272ms/step - loss: 0.3215 - mae: 0.2141 - val_loss: 0.2483 - val_mae: 0.2360
Epoch 3/30
798/798 [==============================] - ETA: 0s - loss: 0.1631 - mae: 0.1904
Epoch 00003: saving model to /colab/My Drive/SelfDrivingCar/model-03-0.19.h5
798/798 [==============================] - 217s 272ms/step - loss: 0.1631 - mae: 0.1904 - val_loss: 0.1853 - val_mae: 0.2305
Epoch 4/30
798/798 [==============================] - ETA: 0s - loss: 0.0999 - mae: 0.1665
Epoch 00004: saving model to /

Below are unnecessary part which is still to be completed. But above code is final and working with great accuracy.


ALL the code below this line should be ignored

In [ ]:
'''
model_clstm = tf.keras.models.Sequential([
tf.keras.layers.LSTM(128, return_sequences= True ),
tf.keras.layers.LSTM(64),
tf.keras.layers.Dense(64, activation = 'relu'),
tf.keras.layers.Dense(32, activation = 'relu'),
tf.keras.layers.Dense(1, activation = 'sigmoid')
])
'''
model.load_weights('/colab/My Drive/Copy of model-08-0.13.h5')


In [ ]:
model_featured = tf.keras.models.Model(
    inputs=model.input,
    outputs=model.get_layer('flatten').output
)

In [ ]:
train_featured = model_featured.predict(train_generator)
test_featured = model_featured.predict(val_generator)

In [ ]:
np.savetxt('/colab/My Drive/ftrainf.csv',train_featured, delimiter=',')
np.savetxt('/colab/My Drive/ftrainf.csv',train_featured, delimiter=',')

In [ ]:
test_featured.shape

(12765, 4096)

In [ ]:
train_featured.shape

(51060, 4096)

In [ ]:
n_steps = 20
temp = train_featured[2:n_steps]
temp = temp.reshape(1,n_steps,temp.shape[1])
for i in range(n_steps,train_featured.shape[0],n_steps):
  temp = np.append(temp,train_featured[i:i+n_steps].reshape(1,n_steps,train_featured.shape[1]),axis = 0)
trainX = temp
temp = test_featured[:n_steps]
temp = temp.reshape(1,temp.shape[0],temp.shape[1])
for i in range(n_steps,test_featured.shape[0],n_steps):
  temp = np.append(temp,test_featured[i:i+n_steps].reshape(1,n_steps,test_featured.shape[1]),axis = 0)
testX = temp

ValueError: ignored

In [ ]:
from keras.layers  import TimeDistributed, MaxPooling2D, LSTM, ConvLSTM2D

In [ ]:

model = Sequential()
model.add(TimeDistributed(Conv2D(36, (5, 5), activation = 'elu', strides = (1, 1), input_shape = (20, 224,224,1))))
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(Conv2D(48, (5, 5), activation = 'elu', strides = (1, 1))) )
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation = 'elu')) )
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation = 'elu', )))
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(Conv2D(256, (3, 3), activation = 'elu')))
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(Conv2D(1024, (3, 3), activation = 'elu', kernel_regularizer=regularizers.l2(0.001))))
model.add(TimeDistributed(Dropout(0.2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(64, return_sequences = True))
model.add(LSTM(128))                          
model.add(Dense(512, activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'linear'))


In [ ]:
# define CNN model
cnn = Sequential()
cnn.add(Conv2D(36, (5, 5), activation = 'elu', strides = (1, 1), input_shape = (224,224,1)))
cnn.add(MaxPooling2D(2,2))
cnn.add(Conv2D(64, (3, 3), activation = 'elu', strides = (1, 1)))
cnn.add(MaxPooling2D(2,2))
cnn.add(Conv2D(128, (3, 3), activation = 'elu', strides = (1, 1)))
cnn.add(MaxPooling2D(2,2))
cnn.add(Conv2D(256, (3, 3), activation = 'elu', strides = (1, 1)))
cnn.add(MaxPooling2D(2,2))
cnn.add(Conv2D(1024, (3, 3), activation = 'elu', strides = (1, 1)))
cnn.add(Flatten())
# define LSTM model
model = Sequential()
model.add(TimeDistributed(cnn))
model.add(LSTM(64, return_sequences = True))
model.add(LSTM(128))
model.add(Dense(512, activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'linear'))

In [ ]:
model.summary()

ValueError: ignored

In [ ]:
    '''
    from keras.layers import Input
    frames = 20
    trailer_input  = Input(shape=(frames, 1, 224, 224)
                    , name='trailer_input')
    
    first_ConvLSTM = ConvLSTM2D(filters=20, kernel_size=(3, 3)
                       , data_format='channels_first'
                       , recurrent_activation='hard_sigmoid'
                       , activation='tanh'
                       , padding='same', return_sequences=True)(trailer_input)
    first_BatchNormalization = BatchNormalization()(first_ConvLSTM)
    first_Pooling = MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_first')(first_BatchNormalization)
    
    second_ConvLSTM = ConvLSTM2D(filters=10, kernel_size=(3, 3)
                        , data_format='channels_first'
                        , padding='same', return_sequences=True)(first_Pooling)
    second_BatchNormalization = BatchNormalization()(second_ConvLSTM)
    second_Pooling = MaxPooling3D(pool_size=(1, 3, 3), padding='same', data_format='channels_first')(second_BatchNormalization)
    
    outputs = [branch(second_Pooling, 'cat_{}'.format(category)) for category in categories]
    
    seq = Model(inputs=trailer_input, outputs=outputs, name='Model ')
    '''

"\nfrom keras.layers import Input\nframes = 20\ntrailer_input  = Input(shape=(frames, 1, 224, 224)\n                , name='trailer_input')\n\nfirst_ConvLSTM = ConvLSTM2D(filters=20, kernel_size=(3, 3)\n                   , data_format='channels_first'\n                   , recurrent_activation='hard_sigmoid'\n                   , activation='tanh'\n                   , padding='same', return_sequences=True)(trailer_input)\nfirst_BatchNormalization = BatchNormalization()(first_ConvLSTM)\nfirst_Pooling = MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_first')(first_BatchNormalization)\n\nsecond_ConvLSTM = ConvLSTM2D(filters=10, kernel_size=(3, 3)\n                    , data_format='channels_first'\n                    , padding='same', return_sequences=True)(first_Pooling)\nsecond_BatchNormalization = BatchNormalization()(second_ConvLSTM)\nsecond_Pooling = MaxPooling3D(pool_size=(1, 3, 3), padding='same', data_format='channels_first')(second_BatchNormalization)\n

In [ ]:
import keras
import tensorflow as tf

filepath = '/colab/My Drive/SelfDrivingCar/model-{epoch:02d}-{val_loss:.2f}.h5'
model_clstm_callbacks =[
  keras.callbacks.EarlyStopping(monitor='val_loss', patience=8),
  keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=True, mode='auto', period=1 ),
  keras.callbacks.CSVLogger('model-logs.csv', separator=',', append=False )
]

In [ ]:
model.compile(loss='mse',
              optimizer='adam',
              metrics=['mae'])

In [ ]:
history_clstm = model.fit(
      train_generator,
        
      epochs=30,
      verbose=1,
      validation_data = val_generator,
      
      callbacks=model_clstm_callbacks)

InvalidArgumentError: ignored